In [2]:
# LSTM-PINN Hybrid Model for Predicting Missing Well Logs
# Author: [Your Name]
# Date: [Date]

import numpy as np
import pandas as pd
import tensorflow as tf
#from tensorflow.keras.models import Model
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.layers import Rescaling, Resizing, CenterCrop
#from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('cleaned_well_data.csv')  

# Define features and targets
input_features = ['SP:1', 'PHIE:1', 'VSH', 'SP:2']
target_logs = ['RES', 'GR', 'RHOB:1']

# Scale features
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X = scaler_x.fit_transform(df[input_features])
y = scaler_y.fit_transform(df[target_logs])

# Reshape input for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM Model Definition
input_layer = Input(shape=(X.shape[1], X.shape[2]))
x = LSTM(64, return_sequences=True)(input_layer)
x = Dropout(0.3)(x)
x = LSTM(32)(x)
x = Dropout(0.2)(x)
output_layer = Dense(3)(x)  # Multivariate output

model = Model(inputs=input_layer, outputs=output_layer)

# Physics-Informed Component (dummy placeholder for physical residuals)


NameError: name 'Input' is not defined

## Physics Informed Neural Network

In [ ]:
def physics_loss(y_true, y_pred):
    # Example: Encourage resistivity predictions to follow smooth transitions
    smooth_penalty = tf.reduce_mean(tf.square(y_pred[:, 0][1:] - y_pred[:, 0][:-1]))
    data_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return data_loss + 0.01 * smooth_penalty

model.compile(optimizer='adam', loss=physics_loss, metrics=['mae'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred_rescaled = scaler_y.inverse_transform(y_pred)
y_test_rescaled = scaler_y.inverse_transform(y_test)

# Visualization
plt.figure(figsize=(12,5))
plt.plot(y_test_rescaled[:, 0], label='True RT')
plt.plot(y_pred_rescaled[:, 0], label='Predicted RT')
plt.title("Resistivity (RT) Prediction")
plt.legend()
plt.show()

# Save the model
model.save('lstm_pinn_well_log_model.h5')
